In [1]:
import networkx as nx
import json
import requests
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
%matplotlib inline

 - These files are downloaded from the wikipedia api

 - https://en.wikipedia.org/w/api.php?format=json&list=allrevisions&arvprop=ids%7Ctimestamp%7Cflags%7Ccomment%7Cuser&arvlimit=500&arvdir=newer&action=query

In [51]:
files = [
    "revision1.json",
    "revision-2.json",
    "revision-3.json",
    "revision-4.json",
    "revision-5.json",
]

In [32]:
graph = nx.DiGraph()
users = []
records=[]

In [36]:
with open(files[0]) as fd:
    data = json.loads(fd.read())
records = data["query"]["allrevisions"]

In [37]:
for record in records:
    revisions = record["revisions"]
    for revision in revisions:
        users.append(revision["user"])

In [38]:
user_list = list(set(users))
len(user_list)

71

In [39]:
url = "https://en.wikipedia.org/w/api.php?\
action=query&format=json&list=allrevisions&arvprop=ids%7Ctimestamp%7Cflags%7Ccomment%7Cuser\
&arvlimit=500&arvdir=newer&arvnamespace=0&arvuser={}"

cat_url = "https://en.wikipedia.org/w/api.php?action=query&prop=categories&pageids={}&format=json"

In [41]:
revisions_per_user = []
s=requests.Session()
for user in user_list:
    response = s.get(url.format(user))
    with response as r:
        content = r.json()
        for i in content["query"]["allrevisions"]:
            # print(i["pageid"])
            # print(requests.get(cat_url.format(i["pageid"])).json())
            try:
                response = s.get(cat_url.format(i["pageid"])).json()
                i["categories"] = [
                    record["title"].split(":")[1]
                    for record in response["query"]["pages"][str(i["pageid"])][
                        "categories"
                    ]
                ]
            except KeyError as e:
                # print("error" + str(e))
                i["categories"] = []
                # print( response["query"]["pages"][str(i["pageid"])])
            #             content["query"]["allrevisions"]["categories"]=[record["title"].split(":")[1] for record in requests.get(cat_url.format(i["pageid"])).json()["query"]["pages"][i["pageid"]]["categories"]]
        revisions_per_user.append({user: content["query"]["allrevisions"]})

In [42]:
with open("rev-wikipedia.json", "w+") as fd:
    fd.write(json.dumps(revisions_per_user))

In [43]:
# transalte usernames to corresponding id's for node2vec
translation_rules = {user: _id for _id, user in enumerate(user_list, start=1)}
translation_rules
translation_rules["Dhcp058.246.lvcm.com"]


def translate(word, rules={}):
    try:
        transalted_val = rules[word]
    except KeyError as error:
        transalted_val = word
        pass
    return transalted_val



translation_rules={}
translate("Dhcp058.246.lvcm.com", translation_rules)

'Dhcp058.246.lvcm.com'

In [44]:
def add_nodes(user_list, attrs):
    for _idx, user in enumerate(user_list, start=1):
        # graph.add_node(_idx, name=user , **attrs)
        # graph.add_node(translate(user, translation_rules), name=user , **attrs)
        graph.add_node(user, name=user, **attrs)
    return graph

In [45]:
import random
def my_hash(arr):
    _hash=0;
    for i in arr:
        _hash+=hash(i)
    random.seed(_hash)
    return random.random()
print(my_hash(["a", "b", "pD950D145.dip.t-dialin.net"]))
print(my_hash(["pD950D145.dip.t-dialin.net", "a", "b"]))

0.4862314456406176
0.4862314456406176


In [46]:
# text = ""
for rev in revisions_per_user:
    #     count = len(graph.nodes())+1
    for user_node_id, user in enumerate(user_list, start=1):
        revisions = rev.get(user)
        if revisions is not None:
            for revision in revisions:
                doc_id = revision["pageid"]
                doc_title = revision["title"]
                # text += ""+str(user)+" : "+str(len(revision["revisions"]))+"\n"
                parent_id = ""
                for _idx, doc in enumerate(revision["revisions"], start=1):
                    # graph.add_node(count, revision_id=doc["revid"], doc_id=doc_id, doc_title=doc_title)
                    # print(revision["categories"])
                    graph.add_node(
                        doc["revid"],
                        revision_id=doc["revid"],
                        doc_id=doc_id,
                        doc_title=doc_title,
                        group=my_hash(revision["categories"]),
                    )
                    if parent_id != doc["parentid"]:
                        # graph.add_edge(count, user_node_id, user_id=user, revision_id=doc["revid"], relation="wasAssociatedWith")
                        #print("source", user)
                        #print(user)
                        graph.add_edge(user,doc["revid"],user_id=user,revision_id=doc["revid"],relation="wasAssociatedWith",group="wasAssociatedWith")
                        #print(user, translate(user, translation_rules))
                        parent_id = doc["revid"]
                        if user == "GaythaLanglois":
                            print(doc["revid"])
                            print(user, "--->", doc["revid"])
                    else:
                        # graph.add_edge(count-1, count, parent_id=doc["parentid"], revision_id=doc["revid"], relation="wasDerviedFrom")
                        graph.add_edge(
                            doc["parentid"],
                            doc["revid"],
                            parent_id=doc["parentid"],
                            revision_id=doc["revid"],
                            relation="wasDerviedFrom",
                            group="wasDerviedFrom",
                        )
                        parent_id = doc["revid"]
                        if user == "GaythaLanglois":
                            print(doc["revid"])
                            print(doc["parentid"], "--->", doc["revid"])
#                     count +=1
# print(text)
# with open("users.txt","w+") as fd:
#                         fd.write(text)
#print(translation_rules)

379849224
GaythaLanglois ---> 379849224
294282
GaythaLanglois ---> 294282
294283
294282 ---> 294283


In [47]:
import json

from networkx.readwrite import json_graph

In [48]:
with open("graph-wikipedia.json", "w+") as fd:
    fd.write(json.dumps(json_graph.node_link_data(graph)))

In [49]:
nx.write_yaml(graph, "graph-wikipedia.yaml")
!code graph-wikipedia.yaml

In [50]:
nx.write_edgelist(graph, "wiki-edgelist")